In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
import os
np.set_printoptions(threshold=np.nan)

TRAIN = 0
EVAL  = 1

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_test = x_test/255

y_train = y_train.reshape([y_train.shape[0]])
y_test = y_test.reshape([y_test.shape[0]])

y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

In [3]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)
flow = datagen.flow(x_train, y_train,batch_size=128)

In [4]:
train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size = 1280).repeat()
train = train.batch(128)
train_itr = train.make_initializable_iterator()
next_batch = train_itr.get_next()

In [5]:
# input
x = tf.placeholder(tf.float32,[None,32,32,3])
y_= tf.placeholder(tf.float32,[None,10])
z = tf.placeholder(tf.float32,[1,32])
# whether is training or not
mode = tf.placeholder(tf.int32,[])

In [6]:
# cross_entropy   
def ce(y_pred, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = labels ,logits = y_pred))

# accuracy    
def acc(y_pred, labels):
    correct_prediction =tf.equal(tf.argmax(y_pred,1), tf.argmax(labels,1))
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32),0)

def noisy_conv2d(inputs,filters,kernel_size,padding,strides,activation,name,scope_name = "conv", noise_on_kernel = True,noise_on_bias = True):
    print("in shape:", inputs.get_shape())
    kernel_shape = [kernel_size,kernel_size,inputs.get_shape().as_list()[-1],filters]
    print("kernel shape:", kernel_shape)
    bias_shape = [filters]
    kernel = tf.Variable(np.random.normal(size = kernel_shape), dtype = tf.float32, name = name + "/kernel")
    bias = tf.Variable(np.random.normal(size = bias_shape), dtype = tf.float32, name = name+"/bias")

    kernel_noise = tf.random_normal(shape = kernel_shape, stddev= 1)
    bias_noise = tf.random_normal(shape = bias_shape, stddev= 0.5)

    if noise_on_kernel:
        kernel = tf.multiply(1 + kernel_noise, kernel)
    if noise_on_bias:
        bias = tf.multiply(1 + bias_noise, bias)
    output = activation(tf.nn.bias_add(tf.nn.conv2d(input = inputs ,filter = kernel,strides = [1, strides, strides, 1], padding = padding, name = name),bias))
    print("out shape:", output.get_shape())
    return output
    
def noisy_dense(inputs,units,activation,name,scope_name = "conv", noise_on_kernel = True, noise_on_bias = True):
    kernel_shape = inputs.get_shape().as_list()[1:] + [units]
    bias_shape = [units]
    kernel = tf.Variable(np.random.normal(size = kernel_shape), dtype = tf.float32, name = name +"/kernel")
    bias = tf.Variable(np.random.normal(size = bias_shape), dtype = tf.float32, name = name + "/bias")

    kernel_noise = tf.random_normal(shape = kernel_shape, stddev= 0.5)
    bias_noise = tf.random_normal(shape = bias_shape, stddev= 0.5)

    if noise_on_kernel:
        kernel = tf.multiply(1 + kernel_noise, kernel)
    if noise_on_bias:
        bias = tf.multiply(1 + bias_noise, bias)
    return activation(tf.nn.bias_add(tf.matmul(inputs,kernel),bias))

In [7]:
with tf.variable_scope("conv") as scope:
    prev_layer = x
    prev_layer = noisy_conv2d(
        inputs = prev_layer,
        filters = 64,
        kernel_size = 3,
        padding = 'VALID',
        strides = 1,
        activation =tf.nn.relu,
        name = 'conv2d_1',
        scope_name = "conv"
    )
    prev_layer = noisy_conv2d(
        inputs = prev_layer,
        filters = 64,
        kernel_size = 3,
        padding = 'VALID',
        strides = 1,
        activation =tf.nn.relu,
        name = 'conv2d_2',
        scope_name = "conv"
    )
    prev_layer = tf.layers.max_pooling2d(
        inputs = prev_layer,
        pool_size = 2,
        strides = 2
    )


    prev_layer = noisy_conv2d(
        inputs = prev_layer,
        filters = 128,
        kernel_size = 3,
        padding = 'VALID',
        strides = 1,
        activation =tf.nn.relu,
        name = 'conv2d_3',
        scope_name = "conv"
    )
    prev_layer = noisy_conv2d(
        inputs = prev_layer,
        filters = 128,
        kernel_size = 3,
        padding = 'VALID',
        strides = 1,
        activation =tf.nn.relu,
        name = 'conv2d_4',
        scope_name = "conv"
    )
    prev_layer = tf.layers.max_pooling2d(
        inputs = prev_layer,
        pool_size = 2,
        strides = 2
    )

    prev_layer = tf.contrib.layers.flatten(prev_layer)
    flat = prev_layer

    prev_layer = noisy_dense(
        inputs = prev_layer,
        units = 256,
        activation = tf.nn.relu,
        name = 'dense_1',
        scope_name = "conv"
    )


    prev_layer = noisy_dense(
        inputs = prev_layer,
        units = 256,
        activation = tf.nn.relu,
        name = 'dense_2',
        scope_name = "conv"
    )

    logits = noisy_dense(
        inputs = prev_layer,
        units = 10,
        activation = lambda t:t,
        name = 'dense_3',
        scope_name = "conv"
    )

in shape: (?, 32, 32, 3)
kernel shape: [3, 3, 3, 64]
out shape: (?, 30, 30, 64)
in shape: (?, 30, 30, 64)
kernel shape: [3, 3, 64, 64]
out shape: (?, 28, 28, 64)
in shape: (?, 14, 14, 64)
kernel shape: [3, 3, 64, 128]
out shape: (?, 12, 12, 128)
in shape: (?, 12, 12, 128)
kernel shape: [3, 3, 128, 128]
out shape: (?, 10, 10, 128)


In [ ]:
pred = tf.nn.softmax(logits)
loss=ce(logits, y_)
accuracy=acc(pred, y_)

theta_conv = tf.trainable_variables("conv")
conv_solver = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss = loss)

sess=tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(train_itr.initializer)



In [ ]:
saver = tf.train.Saver(var_list = theta_conv)
saver.restore(sess,"new_pretrained/model")
i = 0
for batch in flow:
    if i >= 20000:
        break
    if i%100==0:
        acc_batch, loss_batch = sess.run([accuracy, loss], feed_dict={x: batch[0], 
                                                                      y_: batch[1],
                                                                      mode : EVAL
                                                                     }
                                        )
        print("Step: %s, Acc: %s, Loss: %s"%(i,acc_batch, loss_batch))
        
    sess.run(conv_solver, feed_dict={x: batch[0],
                                    y_: batch[1],
                                    mode: TRAIN})
    i+=1

INFO:tensorflow:Restoring parameters from new_pretrained/model
Step: 0, Acc: 0.1796875, Loss: 6.4133463
Step: 100, Acc: 0.1484375, Loss: 2.279562
Step: 200, Acc: 0.1953125, Loss: 2.4492877
Step: 300, Acc: 0.1875, Loss: 2.1257944
Step: 400, Acc: 0.296875, Loss: 1.9597675
Step: 500, Acc: 0.171875, Loss: 2.0537493
Step: 600, Acc: 0.203125, Loss: 2.162696
Step: 700, Acc: 0.25, Loss: 1.9099312
Step: 800, Acc: 0.3125, Loss: 1.8870727
Step: 900, Acc: 0.3359375, Loss: 1.830172
Step: 1000, Acc: 0.3203125, Loss: 1.946311
Step: 1100, Acc: 0.4609375, Loss: 1.6279942
Step: 1200, Acc: 0.3046875, Loss: 1.929491
Step: 1300, Acc: 0.3046875, Loss: 1.9033635
Step: 1400, Acc: 0.453125, Loss: 1.6267785
Step: 1500, Acc: 0.4140625, Loss: 1.5135903
Step: 1600, Acc: 0.3046875, Loss: 1.9558418
Step: 1700, Acc: 0.28125, Loss: 2.0203314
Step: 1800, Acc: 0.5078125, Loss: 1.4898994
Step: 1900, Acc: 0.375, Loss: 1.7646885
Step: 2000, Acc: 0.3671875, Loss: 1.7579997
Step: 2100, Acc: 0.4296875, Loss: 1.5503488
Step: 2

In [ ]:
FINAL_ACC=0.
for i in range(0,10):
    FINAL_ACC+=0.1*sess.run(accuracy, feed_dict={x: x_test[i*1000:(i+1)*1000], y_: y_test[i*1000:(i+1)*1000], mode: EVAL})   
print("Final accuracy on test set:", FINAL_ACC)

In [ ]:
X = None
for i in range(0,1):
    if i == 0:
        X = sess.run(flat, feed_dict={x: x_test[i*1000:(i+1)*1000], y_: y_test[i*1000:(i+1)*1000], mode: EVAL}) 
    else:
        X = np.concatenate((X,sess.run(flat, feed_dict={x: x_test[i*1000:(i+1)*1000], y_: y_test[i*1000:(i+1)*1000], mode: EVAL})), axis = 0)
    print(X.shape)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
colors = [i*25 for i in np.argmax(y_test[0:1000], axis = 1)]

embedding = TSNE(n_components=2).fit_transform(X)
plt.scatter(embedding[:,0],embedding[:,1], c = colors )

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
colors = [i*25 for i in np.argmax(y_test[0:1000], axis = 1)]
#print(colors)
embedding = PCA(n_components=2).fit_transform(X)
plt.scatter(embedding[:,0],embedding[:,1], c = colors )

In [ ]:

test_pred = sess.run(pred, feed_dict={x: x_test[0:1], y_: y_test[0:1], mode: EVAL}) 
test_logits = sess.run(logits, feed_dict={x: x_test[0:1], y_: y_test[0:1], mode: EVAL})
print(test_logits)
print(test_pred)
print(y_test[0])

In [ ]:
test_pred = sess.run(pred, feed_dict={x: x_test[0:1], y_: y_test[0:1], mode: EVAL}) 
test_logits = sess.run(logits, feed_dict={x: x_test[0:1], y_: y_test[0:1], mode: EVAL})
print(test_logits)
print(test_pred)
print(y_test[0])